<a href="https://colab.research.google.com/github/zhangxs131/Spider/blob/main/%E5%8D%A1%E5%86%85%E5%9F%BAspider.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#卡内基spider 

网站  https://carnegieendowment.org/ 卡内基国际和平基金会 

爬取只能用selenium从首页进入，因为直接通过url访问搜索页面会被拒绝

In [1]:
#selenium 和 driver安装
!apt update
!apt install chromium-chromedriver
!pip install selenium

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.launchpad

In [14]:
keywords=['Ukraine','Russia']
max_pages=10

ori_url='https://carnegieendowment.org'
start_time='2022-03-09 23:00:00'
end_time='2022-03-13 00:00:01'

In [15]:
import requests
from lxml import etree

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) '
                        'Chrome/51.0.2704.63 Safari/537.36'}

def get_sampler(sample):
  try:
    page_res = requests.get(sample['url'], headers = headers, timeout = 30)
    page_res.raise_for_status()
    page_res.encoding = page_res.apparent_encoding

  except:
    print('error in geting text pages')
    return {}
  t=etree.HTML(page_res.text)
  try:
    sample['text']=(' ').join(t.xpath('//div[@class="article-body"]/p//text()'))
  except:
    return {}
  return sample

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
import json
import datetime

#配置option
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

browser= webdriver.Chrome(options=options)

#存储数据
result={}
sum=0

# 搜索不同关键词
for keyword in keywords:
  print('__________under keyword {}______________'.format(keyword))

  data=[]
  browser.get("https://carnegieendowment.org/")

  time.sleep(2)
  print(browser.title)

  #点击搜索框
  search_a=browser.find_element(by=By.XPATH,value='//a[@class="menu-trigger main-search-trigger header-navigation-trigger"]')
  search_a.click()

  #选择搜索框并输入关键词
  WebDriverWait(browser, 15).until(EC.presence_of_all_elements_located((By.XPATH, '//input[@name="qry"]')))
  input=browser.find_element(by=By.XPATH,value='//input[@name="qry"]')
  input.send_keys(keyword)
  input.send_keys(Keys.ENTER)

  time.sleep(3)
  print(browser.title)
  print('search_url',browser.current_url)

  #选择按时间排序
  WebDriverWait(browser, 15).until(EC.presence_of_all_elements_located((By.XPATH, '//ul[@class="nav nav-tabs control-tabs"]/li')))
  t_datasort=browser.find_elements(by=By.XPATH,value='//ul[@class="nav nav-tabs control-tabs"]/li')
  datasort=t_datasort[0].find_element(by=By.XPATH,value='./a')
  datasort.click()



  #选择文章类型
  WebDriverWait(browser, 15).until(EC.presence_of_all_elements_located((By.XPATH, '//a[@class="accordion-trigger--alt arrow-down-link border-radius"]')))
  print('date_sort_search_url',browser.current_url)
  document_type=browser.find_elements(by=By.XPATH,value='//a[@class="accordion-trigger--alt arrow-down-link border-radius"]')[0]
  document_type.click()

  #这里只选择了article类型，需要增加其他类型文章，并编写解析函数，否则无数据
  WebDriverWait(browser, 15).until(EC.presence_of_all_elements_located((By.XPATH, '//li[@class="search-result-filters__filter"]')))
  article_type=browser.find_elements(by=By.XPATH,value='//li[@class="search-result-filters__filter"]')[0].find_element(by=By.XPATH,value='./a')
  article_type.click()
  time.sleep(2)

  #获取文章list clearfix

  #不同页数
  for i in range(max_pages):
    WebDriverWait(browser, 15).until(EC.presence_of_all_elements_located((By.XPATH, '//li[@class="clearfix"]')))
    lists=browser.find_elements(by=By.XPATH,value='//li[@class="clearfix"]')

    if len(lists)==0:
      break
    sample={}
    for j in lists:
      #获取信息

      try:
        url_temp=j.find_element(by=By.XPATH,value='./h4/a').get_attribute('href')
        if not url_temp.startswith('http'):
          sample['url']=ori_url+url_temp
        else:
          sample['url']=url_temp
        
        sample['title']=j.find_element(by=By.XPATH,value='./h4/a').text
        time2=j.find_element(by=By.XPATH,value='.//div[@class="gutter-bottom-narrow meta sans-serif uppercase smallest-text tighten-line-height"]').text
        time_temp=(' ').join(time2.split(' ')[-3:])
        sample['time']=str(datetime.datetime.strptime(time_temp,'%B %d, %Y'))

        #判断时间
        if sample['time']<start_time:
          break

        sample=get_sampler(sample)
        if sample=={}:
          continue
        data.append(sample)
        sum+=1
        if sum%5 ==0:
          print('第{}条数据 时间 {}，标题 {}'.format(sum,sample['time'],sample['title']))
      except:
        continue
    
    
    #比较时间
    if sample['time']<start_time:
      print('爬取数据到达{} ，爬取结束'.format(sample['time']))
      break

    #跳到下一页
    try:
      page_label=browser.find_elements(by=By.XPATH,value='//a[@class="page-link"]')[i]
      page_label.click()
    except:
      print('无法跳转到下一页,结束爬虫')
      break


  result[keyword]=data
#结束驱动
browser.quit()

#保存数据
with open('carnegie{}.json'.format(start_time.split(' ')[0]),'w') as f:
  f.write(json.dumps(result,ensure_ascii=False))

print('————————————保存成功，完成爬取——————————————————')